In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("/home/missinguser/CSE/single-stage-opt/hybrid_tokamak/laptop/")
import latexplot
latexplot.set_cmap() 

from simsopt import mhd
from simsopt import geo
from simsopt import objectives
import matplotlib.pyplot as plt
import glob
import py_spec

In [ ]:
# Directories with data
directories = {
    # "rot_ell": "../../freeb_high_aspect_combined",
    # "QA_20": "../../freeb_12-11-16-48-29",
    
    "rot_ell": "../../fixb_12-06-11-52-22",
    "QA_20": "../../fixb_12-20-00-39-00",
}


def get_results(surf, res):
    # Vmec setup
    vmec = mhd.Vmec(verbose=False)
    vmec.boundary = surf
    qs = mhd.QuasisymmetryRatioResidual(vmec, surfaces=np.linspace(0.1, 1, 16), helicity_m=1, helicity_n=0, ntheta=32, nphi=32)

    # Compute quantities
    mean_iota_diff = (vmec.mean_iota() - 0.4384346834911653) ** 2
    f_qs = qs.total()
    r0_diff = (surf.get_rc(0, 0) - 1.0) ** 2
    return {"Directory": key, "Resolution": res, "f_QS": f_qs, "Mean Iota Diff": mean_iota_diff, "R0 Diff": r0_diff}

# Collect results
results = []
for key, directory in directories.items():
    subdirs = glob.glob(f"{directory}/mpol*")
    subdirs[0]

    if directory.startswith("freeb"):
        # Freeboundary
        first_file = glob.glob(f"{subdir}/*_000_000000.sp.h5")[0]
        
        spec_output = py_spec.output.SPECout(first_file)
        surf = mhd.Spec.pyspec_to_simsopt_surf(spec_output, 0)
        results.append(get_results(surf, 0))
    else:
        first_file = glob.glob(f"{subdirs[0]}/input*_000_000000")[0]
        results.append(get_results( geo.SurfaceRZFourier.from_vmec_input(first_file), 0))
    print(first_file)
    for subdir in subdirs:
        res = int(subdir.split("mpol")[-1])
        if directory.startswith("freeb"):
            latest_file = max(glob.glob(f"{subdir}/*_000_*.h5"), key=lambda x: int(x.split("_")[-1].split(".")[0]))
            print(latest_file)
            if latest_file.endswith(".h5"):
                spec_output = py_spec.output.SPECout(latest_file)
                surf = mhd.Spec.pyspec_to_simsopt_surf(spec_output, 0)

                results.append(get_results(surf, res))
        else:
            # Fixboundary
            latest_file = max(glob.glob(f"{subdir}/input*_000_*"), key=lambda x: int(x.split("_")[-1]))
            print(latest_file)
            results.append(get_results( geo.SurfaceRZFourier.from_vmec_input(latest_file), res))

# Convert to DataFrame
df = pd.DataFrame(results)

# Save as LaTeX table
with open("results_table.tex", "w") as f:
    f.write(df.to_latex(index=False, float_format="%.6f"))



In [ ]:

# Plotting f_{QS} vs Resolution
for directory in df["Directory"].unique():
    subdf = df[df["Directory"] == directory].sort_values(by="Resolution")
    plt.semilogy(subdf["Resolution"], subdf["f_QS"], label=directory, marker="o")  # Add marker for distinction

plt.xlabel("Resolution")
plt.ylabel("$f_{QS}$")
plt.legend()
latexplot.savenshow("f_qs_vs_resolution")

In [4]:

# Save as LaTeX table
with open("results_table.tex", "w") as f:
    f.write(df.rename({
        "Directory": "Optimization run",
        "Resolution": "Resolution",
        "Mean Iota Diff": "$(\iota - \iota^*)^2$",
        "f_QS": "$f_QS$",
        "R0 Diff": "$(R_0 -R_0^*)^2$"
    }, axis=1).to_latex(index=False, float_format="%.6f"))

In [ ]:
df.rename({
        "Directory": "Optimization run",
        "Resolution": "Resolution",
        "Mean Iota Diff": "$(\iota - \iota^*)^2$",
        "f_QS": "$f_QS$",
        "R0 Diff": "$(R_0 -R_0^*)^2$"
    }, axis=1)